# Trasformation Framework

## Overview

The ESA Transformation Framework is a component of the
Copernicus Sentinels Collaborative Data Hub Software (DHS) intended to provide
data transformation capabilities via the integration of processing elements
applied on-demand to Copernicus Sentinel products.


Basic features implemented:  
  - Request the list of available Workflow definitions
  - Request the definition of a specific Workflow definition
  - Submit a Transformation Order
  - Request the list of all the Tranformation Orders
  - Monitor a Tranformation Order status

Additional features implemented:
  - Parallel processing
  - Download product from a set of data source (defined in config/hubs_credentials.yaml)
  - Check input workflow parameters before triggering the processing
  - Don't re-submit the same workflow if it is: enqueued, in processing or succefully completed
  - Re-submit on demand failed Transformation Order


## Explore available Workflows 

Once doker compose is running the API endpoints will now be available at http://localhost:8080.

Alternatively to test REST API endpoints can be used : 
 - from a command line interface _curl_ 
 - from a python interface can be used _request_

####  Request the list of available Workflow definition

The following URL can then be visited to display the list of available Workflows:
http://localhost:8080/Workflows

Alternatively:

In [2]:
import requests

res = requests.get('http://localhost:8080/Workflows')
res.json()

{'@odata.context': 'http://localhost:8080/$metadata#Workflows',
 'value': [{'Id': 'sen2cor_l1c_l2a',
   'WorkflowName': 'Sen2Cor_L1C_L2A',
   'Description': 'Product processing from Sentinel-2 L1C to L2A. Processor V2.3.6',
   'InputProductType': 'S2MSI1C',
   'OutputProductType': 'S2MSI2A',
   'WorkflowVersion': '0.1',
   'WorkflowOptions': {'Aerosol_Type': {'Description': 'Default processing via configuration is the rural (continental) aerosol type with mid latitude summer and an ozone concentration of 331 Dobson Units',
     'Type': 'string',
     'Default': 'RURAL',
     'Enum': ['MARITIME', 'RURAL']},
    'Mid_Latitude': {'Description': "If  'AUTO' the atmosphere profile will be determined automatically by the processor, selecting WINTER or SUMMER atmosphere profile based on the acquisition date and geographic location of the tile",
     'Type': 'string',
     'Default': 'SUMMER',
     'Enum': ['SUMMER', 'WINTER', 'AUTO']},
    'Ozone_Content': {'Description': '0: to get the best 

#### Request the definition of a specific Workflow: _sen2cor_l1c_l2a_

The following URL can then be visited to display the _sen2cor_l1c_l2a_ Workflow: 
http://localhost:8080/Workflows('sen2cor_l1c_l2a')

In [1]:
res = requests.get("http://localhost:8080/Workflows('sen2cor_l1c_l2a')")
res.json()

NameError: name 'requests' is not defined

## Transformation order submission _sen2cor_l1c_l2a_


In order to request a new transformation, it is possible to use the command _curl_ in the command-line interface (or _request.post_ in a python interface), passing through a JSON string (or a dictionary) with the full desired configuration, including the workflow id, the product id and the workflow options.

The Transformation Framework will:
- verify the type and range of the options passed in by the User
- submit the processing to the dask workers

- create the processing directory
- download the product from the first hub in the _hubs_credentials.yaml_ that has published the product
- run the plugin workflow
- zip and move the output in the output directory.
- remove the processing directory



#### _sen2cor_ 
The sen2cor_l1c_l2a plugin installed within the TF makes use of Sen2Cor v2.9 tool to convert Sentinel-2 L1C products into L2A output product. It implements classification and atmospheric correction. Currently it is used the Digital Elevation Model of the Shuttle Radar Topography Mission [(SRTM DEM)](http://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/), downloaded by _sen2cor_.


#### List completed Transfromation Orders

In [ ]:
requests.get("http://localhost:8080/TransformationOrders").json()

#### Acquisition on Rome L1C : S2A_MSIL1C_20211216T100421_N0301_R122_T32TQM_20211216T105832

In [ ]:
params_rome = {
    "WorkflowId": "sen2cor_l1c_l2a", 
    "InputProductReference": {
        "Reference": "S2A_MSIL1C_20211216T100421_N0301_R122_T32TQM_20211216T105832",
    }, 
    "WorkflowOptions": {
        "Aerosol_Type": "RURAL", 
        "Mid_Latitude": "AUTO", 
        "Ozone_Content": 0, 
        "Cirrus_Correction": True, 
        "DEM_Terrain_Correction": True,
        "Resolution": 60,
    }
}

order_submission_rome = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params_rome, 
)

order_submission_rome.json()

In [ ]:
order_id_rome = order_submission_rome.json()["Id"]
order_id_rome

#### Acquisition on Paris: S2A_MSIL1C_20211231T105441_N0301_R051_T31UEP_20211231T125354

In [ ]:
params_paris = {
    "WorkflowId": "sen2cor_l1c_l2a", 
    "InputProductReference": {
        "Reference":  "S2A_MSIL1C_20211221T105451_N0301_R051_T31UDQ_20211221T125433", 
        "DataSourceName": "scihub"
    }, 
    "WorkflowOptions": {
        "Aerosol_Type": "MARITIME", 
        "Mid_Latitude": "AUTO", 
        "Ozone_Content": 0, 
        "Cirrus_Correction": False, 
        "DEM_Terrain_Correction": False,
        "Resolution": 60,
    }
}

order_submission_paris = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params_paris, 
)

order_submission_paris.json()

In [ ]:
order_id_paris = order_submission_paris.json()["Id"]
order_id_paris

## Monitor the Tranformation Orders status

The full list of available transformation orders can be retrieved via the following URL:
http://localhost:8080/TransformationOrders

The single order status can be retrieved via the following URL:
http://localhost:8080/TransformationOrders('order_id')

It is also possible to filter accessible orders by status type. In this example, the URL queries the list of completed orders:
http://localhost:8080/TransformationOrders?\$filter=Status%20eq%20'completed'

#### Request list of all Transformation Order

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders")
oders_status.json()

#### Request specific Transfromation Oreder

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders('6b52eb5668797b145d483c5ec77e889b')")
oders_status.json()

## Results on Paris

Read Output filename

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders('{order_id_paris}')")
oders_status.json()

In [ ]:
output_filename = oders_status.json()["OutputFile"]

Unzip Product

In [ ]:
ls ../esa_tf/output/S2A_MSIL2A_20211221T105451_N9999_R051_T31UDQ_20220113T151854.zip

In [ ]:
import glob
import os
! unzip {os.path.join("../esa_tf/output", output_filename)} -d tmp/

rgb_path = glob.glob(f"tmp/{output_filename[:-4]}.SAFE/GRANULE/*/IMG_DATA/R60m/*_TCI_60m.jp2")[0]

Read True Color and Plot

In [ ]:
import rioxarray
from matplotlib import pyplot as plt

rgb = rioxarray.open_rasterio(rgb_path)

plt.figure(figsize=(20, 20))
plt.imshow(rgb.transpose("y", "x", "band"))
plt.title("Rome: Sentinel2 L2A True Color 60m", fontsize="xx-large");

## Additional implemented features


#### Check input workflow parameters before trigger the processing

In [ ]:
params_wrong_options = {
    "WorkflowId": "sen2cor_l1c_l2a", 
    "InputProductReference": {
        "Reference": "S2A_MSIL1C_20211022T062221_N0301_R048_T39GWH_20211022T064133.zip", 
        "DataSourceName": "apihub"
    }, 
    "WorkflowOptions": {
        "Aerosol_Type": "MARITIME", 
        "Mid_Latitude": "AUTO", 
        "Ozone_Content": 0, 
        "Cirrus_Correction": 1, 
        "DEM_Terrain_Correction": True
    }
}

In [ ]:
order_submission_wrong_options = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params_wrong_options, 
)
order_submission_wrong_options.json()

####  Don't re-submit the same workflow if it is queued, in processing or succefully completed

- Submission
- Check submited transformation


In [ ]:
order_submission = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params_rome, 
)

oders_status = requests.get(f"http://localhost:8080/TransformationOrders")
oders_status.json()

#### Resubmit failed workflows

submit workflow

In [ ]:
params = {
    "WorkflowId": "sen2cor_l1c_l2a", 
    "InputProductReference": {
        "Reference": "S2A_MSIL1C_20211022T062221_N0301_R048_T39GWH_20211022T064133.zip", 
        "DataSourceName": "apihub"
    }, 
    "WorkflowOptions": {
        "Aerosol_Type": "MARITIME", 
        "Mid_Latitude": "AUTO", 
        "Ozone_Content": 0, 
        "Cirrus_Correction": True, 
        "DEM_Terrain_Correction": True
    }
}

order_submission = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params, 
)
order_id = order_submission.json()["Id"]
order_submission.json()

monitor failed workflow

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders('{order_id}')")
oders_status.json()

re-submit workflow and monitor

In [ ]:
order_submission = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params, 
)

oders_status = requests.get(f"http://localhost:8080/TransformationOrders")
oders_status.json()

## Results on Rome

Read Output filename

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders('{order_id_rome}')")
oders_status.json()

In [ ]:
output_filename = oders_status.json()["OutputFile"]
output_filename

Unzip product

In [ ]:
! unzip {os.path.join("../esa_tf/output", output_filename)} -d tmp/ 

rgb_path = glob.glob(f"tmp/{output_filename[:-4]}.SAFE/GRANULE/*/IMG_DATA/R60m/*_TCI_60m.jp2")[0]

Read True Color and Plot

In [ ]:
rgb = rioxarray.open_rasterio(rgb_path)

plt.figure(figsize=(20, 20))
plt.imshow(rgb.transpose("y", "x", "band"))
plt.title("Rome: Sentinel2 L2A True Color 60m", fontsize="xx-large");

In [5]:
print(["1,2,", "3", "4"])

['1,2,', '3', '4']


In [12]:
import numpy as np
r= ["1,2,", "3", "4", np.arange(3) ]
print(f"{r}")

['1,2,', '3', '4', array([0, 1, 2])]
